In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten
from keras.optimizers import Adam
from wrappers import make_env
from collections import deque
import random
import numpy as np

In [ ]:
# Deep Q-learning Agent
class DQNAgent:
    def __init__(self, env):
        self.state_size = env.observation_space.shape
        self.action_size = env.action_space.n
        self.memory = deque(maxlen=300000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.02
        self.epsilon_decay = 0.999999
        self.learning_rate = 1e-4
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Conv2D(filters=32,kernel_size=(8,8),strides=(4,4),activation='relu',
                         input_shape=(self.state_size[1],self.state_size[2],self.state_size[0])))
        model.add(Conv2D(filters=64,kernel_size=(4,4),strides=(2,2),activation='relu'))
        model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu'))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse',
                      optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = reward + self.gamma * \
                       np.amax(self.model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1,verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
# initialize gym environment and the agent
env = make_env('PongNoFrameskip-v0')
agent = DQNAgent(env)
env.observation_space.shape

In [ ]:
episodes = 5000
min_memory = 20000
# Iterate the game
for e in range(episodes):

    # reset state in the beginning of each game
    state = env.reset()
    state = np.reshape(state, [1,state.shape[1],state.shape[2],state.shape[0]])
    done = False
    rewards=0
    while not done:
        # turn this on if you want to render
        if (e+1)%10 == 0:
            env.render()

        # Decide action
        action = agent.act(state)

        # Advance the game to the next frame based on the action.
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1,next_state.shape[1],next_state.shape[2],next_state.shape[0]])

        # Remember the previous state, action, reward, and done
        agent.remember(state, action, reward, next_state, done)

        # make next_state the new current state for the next frame.
        state = next_state

        rewards+=reward

        # done becomes True when the game ends
        if done:
            # print the score and break out of the loop
            print("episode: {}/{}, score: {}".format(e, episodes, rewards))
            
    # train the agent with the experience of the episode
    if len(agent.memory) > 20000:
        agent.replay(64)